<center>
<table>
  <tr>
    <td><img src="https://portal.nccs.nasa.gov/datashare/astg/training/python/logos/nasa-logo.svg" width="100"/> </td>
     <td><img src="https://portal.nccs.nasa.gov/datashare/astg/training/python/logos/ASTG_logo.png?raw=true" width="80"/> </td>
     <td> <img src="https://www.nccs.nasa.gov/sites/default/files/NCCS_Logo_0.png" width="130"/> </td>
    </tr>
</table>
</center>

        
<center>
<h1><font color= "blue" size="+3">ASTG Python Courses</font></h1>
</center>

---

<center>
    <h1><font color="red">Simple Regression with Tensorflow</font></h1>
</center>

## <font color="red">Objective</font> <a class="anchor" id="sec_obj"></a>

The objective if this presentation is to create a ML model with TensorFlow for a simple regression problem. We show the steps involved in creating and validating the model. 


__Target audience:__

This document is meant for people who want to start building their own AI/ML models with TensorFlow.

## <font color="red"> Python packages used</font>

- __Matplotlib__: Create visualization.
- __Pandas__: Data (two-dimensional labelled array) manipulation and analysis.
- __Seaborn__: Provide a high-level interface for creating attractive and informative statistical graphics. 
- __Scikit-Learn__:  Provide supervised and unsupervised Machine Learning algorithms.
- __TensorFlow__: Used to to build, train, and evaluate a deep machine learning algorithm based on Neural Networks.

In [ ]:
try:
    import google.colab
    print("Running in Google Colab")
except:
    print("Not running in Google Colab")
else:
    print("Installing modules in Google Colab")
    !pip install seaborn
    !pip install -U scikit-learn

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [ ]:
print(f"Numpy version:      {np.__version__}")
print(f"Pandas version:     {pd.__version__}")
print(f"Seaborn version:    {sns.__version__}")
print(f"TensorFlow version: {tf.__version__}")

## <font color="blue">Problem Statement</font>

We consider the function: <br>
$$
f(x,y) = (1-(x^2 + y^3))e^{-\frac{1}{2}(x^2 + y^2)}
$$
<br>
defined in the domain $D=[-3,3] \times [-3,3]$.
<OL>
<LI> We randomnly select $n$ points in the domain $D$ and compute the function on those points to create a (training) dataset containing $n$ pairs points/values.
<LI> We use the dataset for training a ML algorithm.
<LI> We generate a uniform set of points (testing set) in $D$ to test the algorithm.
</OL>

## <font color="blue">Generating the Data</font>

#### Define the Function

In [ ]:
def ff(x,y):
    return (1-(x**2+y**3))*np.exp(-(x**2+y**2)/2)

#### Create the Data

- We wan to create $50\times 30=1500$ random points in the domain $[-3,3] \times [-3,3]$.

In [ ]:
num_dims = 2
nx = 50
ny = 30
num_points = nx * ny

# Boundary of the domain
a_min = -3.0
a_max = 3.0

In [ ]:
X = np.random.uniform(a_min, a_max, (num_points, num_dims))

In [ ]:
X.shape

In [ ]:
X[0:9,:]

We determine the value of the function:

In [ ]:
z = ff(X[:,0], X[:,1])

In [ ]:
z.shape

In [ ]:
z[0:9]

Now we can create a Pandas DataFrame:

In [ ]:
data = pd.DataFrame({"x": X[:,0], "y": X[:,1], 
                           "TargetValues": z[:]})

In [ ]:
data.head(9)

## <font color="blue">Data Gathering and Basic Analyses</font>

#### Splitting the data into training and testing sets
- We split the data into training and testing sets. 
- We train the model with 80% of the samples and test with the remaining 20%. 
- We do this to assess the model’s performance on unseen data.

In [ ]:
xy_df = data.drop('TargetValues', axis = 1)
z_df = data['TargetValues']

In [ ]:
xy_df

In [ ]:
z_df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(xy_df, 
                                                    z_df, 
                                                    test_size=0.2, 
                                                    random_state=42)

In [ ]:
print(f"Train features shape: {X_train.shape}")

In [ ]:
print(f"Test features shape: {X_test.shape}")

In [ ]:
y_train

#### Plot the data to be trained

In [ ]:
fig = plt.figure()#.gca(projection='3d');
threedee = fig.add_subplot(projection='3d')
threedee.scatter(X_train['x'], X_train['y'], 
                 y_train);
threedee.set_xlabel('x');
threedee.set_ylabel('y');
threedee.set_zlabel('f(x,y)');
plt.show();

In [ ]:
sns.kdeplot(x=X_train['x'], y=X_train['y'], 
            cmap="Blues", shade=True, bw_adjust=.5)

#### Check the overall statistics

In [ ]:
stats_train = X_train.describe()
stats_train

#### Display the joint distribution of the columns from the training set

In [ ]:
sns.pairplot(X_train, diag_kind="kde");

<font color="blue">Add noise in the training targets</font>

- The function `ff` is smooth.
- We want to add noise to the targets.
- We consider as noise a Gaussian normal distribution with `noise_mean` as mean and `noise_std` as standard deviation.

In [ ]:
n_train = y_train.shape[0]
n_train

In [ ]:
y_train

In [ ]:
noise_mean = 0.0
noise_std  = 1.0e-2
noise = np.random.normal(noise_mean, noise_std, n_train)
noise.shape

In [ ]:
#y_train = y_train + noise

In [ ]:
y_train

## <font color="blue">Normailized the Data</font>

- In general, variables may not be a similar scale. High values would gain more importance in any distance-based calculations. 
- It is good practice to normalize features that use different scales and ranges. 
- Although the model might converge without feature normalization, it makes training more difficult, and it makes the resulting model dependent on the choice of units used in the input.

In [ ]:
stats_train = X_train.describe().transpose()
stats_train

In [ ]:
def normalize_data(x):
    """
       Normalize the data
    """
    return (x - stats_train['mean']) / stats_train['std']

**Normalize the data that will be used to train the model**

In [ ]:
X_train_normed = normalize_data(X_train)

In [ ]:
X_train_normed

**We also need to normalize the test dataset by projecting it into the same distribution that the model has been trained on**

In [ ]:
X_test_normed = normalize_data(X_test)

<font color="blue">**The same normalization will have to be applied to any other data used in this model.**</font>

## <font color="blue">Build the model</font>

#### Determining the parameters of the Neural Network

- We have two (2) features and one (1) target.
   - The Neural Network will have `input_dim=2` input neurons and `output_dim=1` output neuron.
- In each hidden layer:
   - We want the number of hidden neurons to be less than twice the size of the input layer.
   - We select three (3) neurons in the first hidden layer and three (3) in the second.

__Dataset parameters__

These parameters are defines by the dataset used:

- number of features
- number of outputs

In [ ]:
input_dim = len(X_train.keys())
output_dim = 1

__Model parameters__

- batch size
- number of epochs
- learning rate (optimizer steps)

In [ ]:
batch_size = 32 # not used here
num_epochs = 100
learning_rate = 0.001

In [ ]:
num_hidden_nodes = 3

#### Instantiate a sequential model using `keras`

We create a sequential Neural Network:

- The model expects rows of data with `input_dim` variables (the `input_shape=[input_dim]` argument)
- The first hidden layer has `num_hidden_nodes` nodes and uses the `relu` activation function.
- The second hidden layer has `num_hidden_nodes` nodes and uses the `relu` activation function.
- The output layer has one node and uses no activation function.

In [ ]:
model = keras.Sequential([
             layers.Dense(num_hidden_nodes, activation=tf.nn.relu, 
                          input_shape=[input_dim]),
             layers.Dense(num_hidden_nodes, activation=tf.nn.relu),
             layers.Dense(output_dim) ])

The above model creation can also be written as:

```python
model = keras.Sequential()
model.add(layers.Dense(num_hidden_nodes, activation=tf.nn.relu, 
                       input_shape=[input_dim]))
model.add(layers.Dense(num_hidden_nodes, activation=tf.nn.relu))
model.add(layers.Dense(output_dim))
```

Dense layers represent a function that maps the input tensor `x` to an output tensor `y` via the equation `y = Ax + b` where `A` (the kernel) and `b` (the bias) are parameters of the dense layer.

![nn](../images/tensorflow_nn.png)

#### Compile the model
- Once you have specified the architecture of the network, you need to specify the method for back-propagation by choosing an optimizer and specify the loss.
- Compiling the model uses the efficient numerical libraries (Theano or TensorFlow) in the background.

Define the optimizer:

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate)

Required to provide a loss function and an optimizer: 
- We are asking the network to use the `rmsprop` optimizer to change weights in such a way that the loss `mse` (mean squared error) is minimized at each iteration.

In [ ]:
model.compile(loss = 'mse',
              optimizer = optimizer,
              metrics = ['mae', 'mse'])

#### Inspect the model

`model.summary()` is a useful method if you want to get an overview of your model and see the total number of parameters.
It prints:

- Name and type of all layers in the model.
- Output shape for each layer.
- Number of weight parameters of each layer.
-  If the model has general topology, the inputs each layer receives
- The total number of trainable and non-trainable parameters of the model.



In [ ]:
model.summary()

In [ ]:
import tensorflow.keras.backend as K

trainable_count = np.sum([K.count_params(w) for w in model.trainable_weights])
non_trainable_count = np.sum([K.count_params(w) for w in model.non_trainable_weights])

print('Total params: {:,}'.format(trainable_count + non_trainable_count))
print('Trainable params: {:,}'.format(trainable_count))
print('Non-trainable params: {:,}'.format(non_trainable_count))

[Let](https://towardsdatascience.com/counting-no-of-parameters-in-deep-learning-models-by-hand-8f1716241889):

- **i**: input size (2 in this case)
- **h**: size of hidden layers (3, 3 here)
- **o**: output size (1 in this case)

We have:
 
 $$
 \begin{align*}
 num\_params &=& connections\_between\_layers + biases\_in\_every\_layer \\
               &=& (i \times h + h \times o) + (h+o) \\
               &=& (2\times 3 + 3 \times 3 + 3 \times 1) + (3 + 3 + 1) \\
               &=& (2 \times 3 + 3) + (3 \times 3 + 3) + (3 \times 1 + 1) \\
               &=& 9 + 12 + 3 \\
               &=& 25
 \end{align*}
 $$
 
 
 

               
   input = **Input**((None, 2))
   <br>
   dense = **Dense**(3)(input)
      <br>
   dense = **Dense**(3)(dense)
    <br>
  output = **Dense**(1)(dense)
   <br>
   model = Model(input, output)

#### Try the model

10 samples from the training data and call `model.predict`.

In [ ]:
sample_input = X_train_normed[:10]

In [ ]:
sample_result = model.predict(sample_input)

In [ ]:
print(sample_result)

It seems to be working, and it produces a result of the expected shape and type.

## <font color="blue">Train the model</font>

- We use the `fit` method to train the model.
- A callback function is applied at given stages of the training procedure. We use it to get a view on internal states and statistics of the model during training.

In [ ]:
# Display training progress by printing a 
# single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
      def on_epoch_end(self, epoch_id, logs):
          if epoch_id % 100 == 0: 
             print('')
          print('.', end='')

In [ ]:
history = model.fit(X_train_normed, y_train,    
                    epochs=num_epochs, verbose=1, 
                    callbacks=[PrintDot()])

#### Visualize the model's training progress

In [ ]:
# Use the stats stored in the history object.
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
print(history.history.keys())

In [ ]:
keys = list(history.history.keys())
keys

In [ ]:
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [Target]')
    plt.plot(hist['epoch'], hist[keys[1]], label='Train Error')
    plt.legend()
    plt.ylim([min(hist[keys[1]]) ,max(hist[keys[1]])])

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [$Target^2$]')
    plt.plot(hist['epoch'], hist[keys[2]], label='Train Error')            
    plt.legend()
    plt.ylim([0,max(hist[keys[2]])])

plot_history(history)

## <font color="blue">Evaluate the Model on Test Data</font>

**Compute the Scores**

In [ ]:
loss, mae, mse = model.evaluate(X_test_normed, y_test, verbose=1)

In [ ]:
print(f"Test data:")
print(f"\t loss = {loss} \n\t  mae = {mae} \n\t  mse = {mse}")

**Make Prediction**

In [ ]:
y_test_pred = model.predict(X_test_normed).flatten()

#### Model Evaluation

In [ ]:
rmse = np.sqrt(np.mean((y_test - y_test_pred) ** 2))

print(f"The model performance for test set")
print(f"----------------------------------")
print(f"Root Mean Squared Error: {rmse}")

#### Do the 45-degree plot

In [ ]:
plt.scatter(y_test, y_test_pred);
plt.xlabel('True Values');
plt.ylabel('Predictions');
plt.axis('equal');
plt.axis('square');
plt.xlim([0,plt.xlim()[1]]);
plt.ylim([0,plt.ylim()[1]]);
_ = plt.plot([-100, 100], [-100, 100]);

**Error Distribution**

In [ ]:
sns.distplot(y_test_pred - y_test);

#### Plotting Function Using Predicted Values

In [ ]:
fig = plt.figure()#.gca(projection='3d');
threedee = fig.add_subplot(projection='3d')
threedee.scatter(X_test['x'], X_test['y'], y_test_pred);
threedee.set_xlabel('x');
threedee.set_ylabel('y');
threedee.set_zlabel('f(x,y)');
plt.show();

# <font color="red">Exercise</font> <a class="anchor" id="sec_tf_ex"></a>

- You noticed that the model was underfitting. 
- Recreate the model by increasing the number of nodes in the two hidden layers till you get acceptable results without overfitting the model.
- Also consider changing the batch size.

# <font color="red">Useful references</font>

- <a href="https://www.mygreatlearning.com/blog/what-is-tensorflow-machine-learning-library-explained/">What is TensorFlow? The Machine Learning Library Explained</a>
- <a href="https://www.tensorflow.org/tutorials/keras/regression">Basic regression: Predict fuel efficiency</a>
- <a href="https://stackabuse.com/tensorflow-2-0-solving-classification-and-regression-problems/">Tensorflow 2.0: Solving Classification and Regression Problems</a>
- <a href="https://www.toptal.com/machine-learning/tensorflow-machine-learning-tutorial">Getting Started with TensorFlow: A Machine Learning Tutorial</a>
- <a href="https://sebastianraschka.com/faq/docs/tensorflow-vs-scikitlearn.html">What is the main difference between TensorFlow and scikit-learn?</a>
- <a href="https://adventuresinmachinelearning.com/python-tensorflow-tutorial/">Python TensorFlow Tutorial – Build a Neural Network</a>
- <a href="https://steadforce.com/en/first-steps-tensorflow-part-3/">A simple neural network with TensorFlow</a>